# ABECTO Tutorial

ABECTO is running as a REST service in the background.

The script `abecto.R` provides handy functions to use the service, hidding the raw HTTP requests.

In [ ]:
source("abecto.R")

First, the service needs to be startet. **This might take a few seconds.**

In [ ]:
start()

In [ ]:
project <- createProject("Sample")

In [ ]:
listProjects()

In [ ]:
getProject(project)

In [ ]:
kb1 <- createKnowledgebase(project)

In [ ]:
getKnowledgebase(kb1)

In [ ]:
deleteKnowledgebase(kb1)

In [ ]:
listKnowledgebases()
listKnowledgebases(project)